In [1]:
import torch
import torchtext
import torch.nn as nn 
import torch.nn.functional as F

In [2]:
import numpy as np 
import pandas as pd 
import sklearn
import scipy.io as scio 

import seaborn as sns
import matplotlib.pyplot as plt 

import time

In [3]:
DATA_PATH = 'Data/GDB'
DEVICE = torch.device('cuda')

In [4]:
stft = scio.loadmat(DATA_PATH + '/SE_STFT_GDB.mat')
mfcc = scio.loadmat(DATA_PATH + '/SE_MFCC_GDB.mat')

#get data
sdb = stft['DB']
mdb = mfcc['DB']

#get feature
sdata = sdb[:, 1:]
mdata = mdb[:, 1:]

#get target label
labels = sdb[:, 0]

#make sure that the labels in stft_gd and mfcc_gd are the same
assert sdb[:, 0].all() == mdb[:, 0].all()

In [5]:
# (s_r, s_c) = sdata.shape
# s_data = sdata.reshape(s_r, 1, s_c)

In [6]:
# (m_r, m_c) = mdata.shape
# m_data = mdata.reshape(m_r, 1, m_c)

In [7]:
# data = np.concatenate((s_data, m_data), axis=1)

In [8]:
data = np.concatenate((sdata, mdata), axis=1)

In [9]:
data.shape

(2150, 650)

In [10]:
data = data.astype(np.float32)
labels = labels.astype(np.int32)

In [11]:
#先试一下conv network
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()

        self.dense1 = nn.Linear(650, 64)
        self.drop1 = nn.Dropout(0.2)
        self.dense2 = nn.Linear(64, 32)
        self.drop2 = nn.Dropout(0.2)
        self.dense3 = nn.Linear(32, 16)
        self.out = nn.Linear(16, 10)


    def forward(self, x):
        x = self.dense1(x)
        x = F.relu(x)
        x = self.drop1(x)
        x = self.dense2(x)
        x = F.relu(x)
        x = self.drop2(x)
        x = self.dense3(x)
        x = F.relu(x)
        
        output = self.out(x)
        output = F.softmax(output)

        return output

In [12]:
dataset = torch.utils.data.TensorDataset(torch.Tensor(data), torch.Tensor(labels))

In [13]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

In [14]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [15]:
train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=32)
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=True, batch_size=32)

In [25]:
#train model
model = ConvNet().to(DEVICE)
criterion = torch.nn.CrossEntropyLoss().to(DEVICE)
optim = torch.optim.Adam(model.parameters())

In [26]:
print(model)

ConvNet(
  (dense1): Linear(in_features=650, out_features=64, bias=True)
  (drop1): Dropout(p=0.2, inplace=False)
  (dense2): Linear(in_features=64, out_features=32, bias=True)
  (drop2): Dropout(p=0.2, inplace=False)
  (dense3): Linear(in_features=32, out_features=16, bias=True)
  (out): Linear(in_features=16, out_features=10, bias=True)
)


In [27]:
model.train()
N_EPOCHS = 64

acces = []
losses = []
for epoch in range(N_EPOCHS):
    print('Begin to train epoch ', epoch)
    epoch_loss = 0
    epoch_acc = 0
    start_time = time.time()
    length = len(train_dataset)

    for data, label in train_loader:
        data = data.cuda()
        label = label.long().cuda()

        output = model(data)

        optim.zero_grad()

        loss = criterion(output, label)
        accuracy = torch.eq(torch.argmax(output, dim=1), label).sum().float().item()

       # print(torch.argmax(output, dim=1))
        
        #print(label)


        loss.backward()
        optim.step()

        epoch_loss += loss
        epoch_acc += accuracy

    print(epoch_acc/length)

    losses.append(epoch_loss/length)
    acces.append(epoch_acc/length)

Begin to train epoch  0
0.14069767441860465
Begin to train epoch  1
0.21104651162790697
Begin to train epoch  2
0.2988372093023256
Begin to train epoch  3
0.41627906976744183
Begin to train epoch  4
0.4982558139534884
Begin to train epoch  5
0.588953488372093
Begin to train epoch  6
0.6284883720930232
Begin to train epoch  7
0.6674418604651163
Begin to train epoch  8
0.7087209302325581
Begin to train epoch  9
0.7122093023255814
Begin to train epoch  10
0.733139534883721
Begin to train epoch  11
0.7604651162790698
Begin to train epoch  12
0.7715116279069767
Begin to train epoch  13
0.7854651162790698
Begin to train epoch  14
0.7808139534883721
Begin to train epoch  15
0.7988372093023256
Begin to train epoch  16
0.8087209302325581
Begin to train epoch  17
0.8063953488372093
Begin to train epoch  18
0.8162790697674419
Begin to train epoch  19
0.8232558139534883
Begin to train epoch  20
0.833139534883721
Begin to train epoch  21
0.8383720930232558
Begin to train epoch  22
0.847093023255813

In [18]:
a = torch.randn(3)
b = torch.randn(3)

In [19]:
criterion(a, b)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)